In [1]:
import pandas as pd
import json

In [2]:
hc3_df = pd.read_json(path_or_buf="../datasets/reddit_eli5.jsonl", lines=True)
hc3_df

,question,human_answers,chatgpt_answers,index
0,"Why is every book I hear about a "" NY Times # ...","[Basically there are many categories of "" Best...",[There are many different best seller lists th...,NaN
1,"If salt is so bad for cars , why do we use it ...",[salt is good for not dying in car crashes and...,[Salt is used on roads to help melt ice and sn...,NaN
2,Why do we still have SD TV channels when HD lo...,[The way it works is that old TV stations got ...,[There are a few reasons why we still have SD ...,NaN
3,Why has nobody assassinated Kim Jong - un He i...,[You ca n't just go around assassinating the l...,[It is generally not acceptable or ethical to ...,NaN
4,How was airplane technology able to advance so...,[Wanting to kill the shit out of Germans drive...,[After the Wright Brothers made the first powe...,NaN
...,...,...,...,...
17107,Why is the general Xbox One reveal feedback ne...,[A few items people do n't seem to like : 1 . ...,[It's difficult to say exactly why the general...,NaN
17108,If infrared is thermal why does n't my remote ...,"[It 's very , very low power . Around the orde...",[Infrared light is a type of light that we can...,NaN
17109,Why are cars all about their brake - horse pow...,[Brake horsepower has nothing to do with brake...,"[Cars are not just about how fast they can go,...",NaN
17110,"What does a CO^2 scrubber do , and how does it...",[A [ scrubber ] ( URL_0 ) is a engineering pro...,[A CO2 scrubber is a device that removes carbo...,NaN


In [3]:
# drop index col which is all NaN
processed_hc3_df = hc3_df.drop(["index"], axis=1)

# keep rows with non empty chatgpt ans
has_chatgpt_ans = processed_hc3_df["chatgpt_answers"].map(lambda ans_lst:ans_lst!=[])
processed_hc3_df = processed_hc3_df[has_chatgpt_ans]

#Need to expand the answers and chatgpt ans -- as for one question, there can be several human answers obtained
row_list = []
for index, row in processed_hc3_df.iterrows():
  human_ans_list = row["human_answers"]
  chatgpt_ans_list = row["chatgpt_answers"]
  # note that sometimes there can be 2 human ans, and 1 chatgpt ans for e.g. 
  num_chatgpt_ans = len(chatgpt_ans_list)
  for i in range(len(human_ans_list)):
    human_ans = human_ans_list[i]
    if i >= num_chatgpt_ans:
      chatgpt_ans = chatgpt_ans_list[num_chatgpt_ans-1]
    else:
      chatgpt_ans = chatgpt_ans_list[i]
    row_list.append([row["question"],human_ans,chatgpt_ans])

processed_hc3_df = pd.DataFrame(row_list, columns = ["question", "human_answer", "chatgpt_answer"])
processed_hc3_df.head()

,question,human_answer,chatgpt_answer
0,"Why is every book I hear about a "" NY Times # ...","Basically there are many categories of "" Best ...",There are many different best seller lists tha...
1,"Why is every book I hear about a "" NY Times # ...","If you 're hearing about it , it 's because it...",There are many different best seller lists tha...
2,"Why is every book I hear about a "" NY Times # ...","One reason is lots of catagories . However , h...",There are many different best seller lists tha...
3,"If salt is so bad for cars , why do we use it ...",salt is good for not dying in car crashes and ...,Salt is used on roads to help melt ice and sno...
4,"If salt is so bad for cars , why do we use it ...","In Minnesota and North Dakota , they tend to u...",Salt is used on roads to help melt ice and sno...


In [7]:
chatgpt_responses = processed_hc3_df.loc[:, ["chatgpt_answer"]]
chatgpt_responses["label"]=1
chatgpt_responses.rename(columns = {'chatgpt_answer':'response'}, inplace = True)
chatgpt_responses.drop_duplicates(inplace=True)
print(chatgpt_responses.shape)
chatgpt_responses.head()

(16122, 2)


,response,label
0,There are many different best seller lists tha...,1
3,Salt is used on roads to help melt ice and sno...,1
6,There are a few reasons why we still have SD (...,1
9,It is generally not acceptable or ethical to a...,1
12,After the Wright Brothers made the first power...,1


In [8]:
human_responses = processed_hc3_df.loc[:, ["human_answer"]]
human_responses["label"]=0
human_responses.rename(columns = {'human_answer':'response'}, inplace = True)
human_responses.drop_duplicates(inplace=True)
print(human_responses.shape)
human_responses.head()

(45509, 2)


,response,label
0,"Basically there are many categories of "" Best ...",0
1,"If you 're hearing about it , it 's because it...",0
2,"One reason is lots of catagories . However , h...",0
3,salt is good for not dying in car crashes and ...,0
4,"In Minnesota and North Dakota , they tend to u...",0


In [9]:
Corpus = pd.concat([chatgpt_responses, human_responses], ignore_index=True)
Corpus.head()

,response,label
0,There are many different best seller lists tha...,1
1,Salt is used on roads to help melt ice and sno...,1
2,There are a few reasons why we still have SD (...,1
3,It is generally not acceptable or ethical to a...,1
4,After the Wright Brothers made the first power...,1


In [10]:
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/daniel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/daniel/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/daniel/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/daniel/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/daniel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
# Step - a : Remove blank rows if any.
Corpus['response'].dropna(inplace=True)
# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
Corpus['response'] = [entry.lower() for entry in Corpus['response']]
# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
Corpus['response']= [word_tokenize(entry) for entry in Corpus['response']]
# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(Corpus['response']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    Corpus.loc[index,'text_final'] = str(Final_words)

In [12]:
Corpus.head()

,response,label,text_final
0,"[there, are, many, different, best, seller, li...",1,"['many', 'different', 'best', 'seller', 'list'..."
1,"[salt, is, used, on, roads, to, help, melt, ic...",1,"['salt', 'use', 'road', 'help', 'melt', 'ice',..."
2,"[there, are, a, few, reasons, why, we, still, ...",1,"['reason', 'still', 'sd', 'standard', 'definit..."
3,"[it, is, generally, not, acceptable, or, ethic...",1,"['generally', 'acceptable', 'ethical', 'advoca..."
4,"[after, the, wright, brothers, made, the, firs...",1,"['wright', 'brother', 'make', 'first', 'powere..."


In [13]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text_final'],Corpus['label'],test_size=0.3)

In [14]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [15]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(Corpus['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [16]:
print(Tfidf_vect.vocabulary_)

{'many': 2696, 'different': 1239, 'best': 427, 'seller': 3929, 'list': 2598, 'publish': 3517, 'various': 4776, 'organization': 3097, 'new': 2968, 'york': 4991, 'time': 4526, 'one': 3064, 'rank': 3576, 'book': 489, 'united': 4701, 'state': 4230, 'base': 378, 'sale': 3854, 'data': 1096, 'number': 3013, 'retailer': 3761, 'newspaper': 2970, 'widely': 4914, 'consider': 922, 'influential': 2269, 'industry': 2255, 'important': 2216, 'note': 2997, 'category': 654, 'country': 1011, 'possible': 3363, 'could': 1006, 'another': 198, 'additionally': 68, 'term': 4455, 'often': 3056, 'use': 4745, 'refer': 3646, 'sell': 3928, 'well': 4895, 'regardless': 3662, 'whether': 4906, 'specific': 4168, 'may': 2734, 'hear': 2061, 'even': 1564, 'specifically': 4169, 'salt': 3856, 'road': 3797, 'help': 2077, 'melt': 2762, 'ice': 2173, 'snow': 4102, 'improve': 2222, 'traction': 4577, 'winter': 4933, 'month': 2865, 'cold': 807, 'outside': 3117, 'water': 4868, 'freeze': 1849, 'make': 2669, 'dangerous': 1091, 'car': 

In [17]:
RBF_SVM = svm.SVC(C=1.0, kernel='rbf')
RBF_SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_RBF_SVM = RBF_SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
# print("SVM Accuracy Score -> ",accuracy_score(predictions_RBF_SVM, Test_Y)*100)

print(classification_report(Test_Y, predictions_RBF_SVM, target_names=['human', 'chatgpt']))

              precision    recall  f1-score   support

       human       0.95      0.98      0.96     13535
     chatgpt       0.94      0.86      0.89      4955

    accuracy                           0.95     18490
   macro avg       0.94      0.92      0.93     18490
weighted avg       0.95      0.95      0.95     18490



In [16]:
SVM = svm.SVC(C=1.0, kernel='linear')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
# print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

print(classification_report(Test_Y, predictions_SVM, target_names=['human', 'chatgpt']))

              precision    recall  f1-score   support

       human       0.95      0.97      0.96     13675
     chatgpt       0.90      0.86      0.88      4815

    accuracy                           0.94     18490
   macro avg       0.93      0.91      0.92     18490
weighted avg       0.94      0.94      0.94     18490



In [15]:
Sigmoid_SVM = svm.SVC(C=1.0, kernel='sigmoid')
Sigmoid_SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_Sigmoid_SVM = Sigmoid_SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
# print("SVM Accuracy Score -> ",accuracy_score(predictions_Sigmoid_SVM, Test_Y)*100)

print(classification_report(Test_Y, predictions_Sigmoid_SVM, target_names=['human', 'chatgpt']))

              precision    recall  f1-score   support

       human       0.95      0.97      0.96     13675
     chatgpt       0.90      0.85      0.87      4815

    accuracy                           0.94     18490
   macro avg       0.92      0.91      0.91     18490
weighted avg       0.93      0.94      0.93     18490



In [18]:
Polynomial_SVM = svm.SVC(C=1.0, kernel='poly', degree=5)
Polynomial_SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_Polynomial_SVM = Polynomial_SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
# print("SVM Accuracy Score -> ",accuracy_score(predictions_Polynomial_SVM, Test_Y)*100)

print(classification_report(Test_Y, predictions_Polynomial_SVM, target_names=['human', 'chatgpt']))

SVM Accuracy Score ->  76.18712817739318


In [18]:
# save the model to disk
import pickle
with open('svm_model.pkl','wb') as f:
    pickle.dump(RBF_SVM,f)